<a href="https://colab.research.google.com/github/Shirouky/lab2-Data-analysis/blob/main/LW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Задание 1

In [ ]:
import requests
from os import makedirs

PAGES_NUMBER = 79
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}
DIR_NAME = "Saved_pages"
makedirs(DIR_NAME, exist_ok=True)

"""
Это алгоритм перебора и скачивания HTML страниц сайта до указанного  количества (PAGES_NUMBER).
"""
for i in range(PAGES_NUMBER):
  url = f"https://biser-optom.ru/category/biser-Preciosa?page={i}"
  response = requests.get(url, headers=HEADERS, timeout=10)

  #Если смогли получить данные с сайта, то сохранение (проверка по коду статуса)
  if response.status_code == 200:
    with open(f"{DIR_NAME}/beads{i}.html", 'w') as f:
        f.write(response.text)
  else:
    print(f"Ошибка: {response.status_code}")

#Задание 2

In [ ]:
"""
Декоратор для преобразования пустых блоков в None
"""
def decorator(func):
  def inner(*args):
    try:
      return func(*args)
    except:
      return None
  return inner

In [ ]:
"""
Декоратор для блоков с числами в int
"""
def digit(func):
  def inner(*args):
    result = func(*args)

    if isinstance(result, str):
      temp = result.split()
      for t in temp:
        try:
          return int(t)
        except ValueError:
          continue

    return result
  return inner

In [ ]:
"""
Класс для парсинга карточки товара
"""
class Card():
  image = ""
  procent = 0
  currency = ""
  price = 0
  old_price = 0
  quantity = 0
  code = ""
  id_web = 0
  id = 0
  url = ""
  type_ = ""
  size = ""
  weight = ""
  color = ""
  card = ""

  def __init__(self, card):
    self.card = card

    self.image = self.set_image()
    self.procent = self.set_procent()
    self.currency = self.set_currency()
    self.price = self.set_price()
    self.old_price = self.set_old_price()
    self.quantity = self.set_quantity()
    self.code = self.set_code()
    self.id_web = self.set_id_web()
    self.url = self.set_url()

    self.set_name()

  @decorator
  def set_name(self):
    name = self.card.find("div", "goods-image").find("img").get("alt")
    temp = name.split()
    if (len(name) < 6):
      name = {}
    else:
      self.type_ = temp[2]
      self.size = temp[3]
      self.weight = self.set_weight(temp[4][:-1])
      self.color =' '.join(temp[6:])

    return name

  @digit
  def set_weight(self, data):
    return data

  @decorator
  def set_image(self):
    return self.card.find("div", "goods-image").find('a').get("href")

  @digit
  @decorator
  def set_procent(self):
    return self.card.find("span", "badge-sales").text

  @decorator
  def set_currency(self):
    return self.card.find("meta").get("content")

  @digit
  @decorator
  def set_price(self):
    return self.card.find("span", itemprop="price").text

  @digit
  @decorator
  def set_old_price(self):
    return self.card.find("span", "old_price").text

  @digit
  @decorator
  def set_quantity(self):
    return self.card.find("div", "availability").find("span").text

  @decorator
  def set_code(self):
    return self.card.find("div", itemprop="description").find("span").text

  @digit
  @decorator
  def set_id_web(self):
    return self.card.find("input", autocomplete="off").get("data-id")

  def set_id(self, id):
    self.id = id

  @decorator
  def set_url(self):
    return self.card.find("meta", itemprop="url").get("content")

  def export(self):
    data = {}
    data["id"] = self.id
    data["id_web"] = self.id_web
    data["code"] = self.code
    data["type"] = self.type_
    data["size"] = self.size
    data["weight"] = self.weight
    data["color"] = self.color
    data["sale"] = self.procent
    data["price"] = self.price
    data["old_price"] = self.old_price
    data["currency"] = self.currency
    data["quantity"] = self.quantity
    data["image"] = self.image
    data["url"] = self.url
    return data

In [ ]:
from bs4 import BeautifulSoup

cards = []
count = 0

for i in range(PAGES_NUMBER):
  with open (f"{DIR_NAME}/beads{i}.html") as html_file:
    page = html_file.read()

  soup_page = BeautifulSoup(page, "html.parser")
  div_blocks = soup_page.find_all("div", "card")[:-1]

  for div_block in div_blocks:
    count += 1
    card = Card(div_block)
    card.set_id(count)
    cards.append(card.export())
div_block

<div class="card" itemscope="" itemtype="http://schema.org/Product"> <div class="goods-image"> <a href="/uploads/goods//S5mdVtM06MnoNJX.jpg" rel="facebox"> <img alt="Бисер Чехия круглый 06/0 50г 68505 медный" class="lazy" data-src="/uploads/goods//c200/thumnail_S5mdVtM06MnoNJX.jpg" itemprop="image"/> </a> <div class="favorite-absolute"> <div class="favorite-wrapper"> <div> <a class="add-to-fa-no-login" data-id="198428" href="" title="В Избранное"> <i class="fa fa-heart-o"></i> </a> </div> </div> </div> </div> <div class="card-body"> <div class="gallery__item__price" itemprop="offers" itemscope="" itemtype="http://schema.org/Offer"> <meta content="RUB" itemprop="priceCurrency"/> <meta content="https://biser-optom.ru/goods/biser-chehiya-kruglyy-06-0-50-g-68505-mednyy" itemprop="url"/> <link href="http://schema.org/InStock" itemprop="availability"/> <div class="price"><span itemprop="price">132</span> ₽  </div> </div> <div class="gallery__item__desc"> <a href="https://biser-optom.ru/goods

In [ ]:
from json import dump

"""
Сохранение полученных данных в файл json
"""
def json_save(output_data):
    with open("beads.json", 'w') as f:
        dump(output_data, f, indent=2, ensure_ascii=False)

    print("Данные сохранены в json")

json_save(cards)

Данные сохранены в json
